In [10]:
import re
import pandas as pd
import yaml
from scripts.utils import create_file_list, deduplicate_highest_attempt

In [5]:
WEEK_NUMBER = 2

In [6]:
# Loading global settings
with open("settings.yaml") as f:
    settings = yaml.safe_load(f)

# Global settings
WEEK = settings["weeks"][WEEK_NUMBER]
STUDENT_SUBMISSION_TEMPLATE = settings["global"]["paths"]["student_submission_template"]
STUDENT_SUBMISSION_JSON_TEMPLATE = settings["global"]["paths"]["student_submission_json_template"]
SUBMISSION_PATH = settings["global"]["paths"]["submissions"]

# Global canvas settings
COURSE_ID = settings["global"]["canvas"]["course_id"]
ASSIGNMENT_ID = WEEK["canvas"]["assignment_id"]
QUIZ_ID = WEEK["canvas"]["quiz_id"]
R_QUIZ_QUESTION_ID = WEEK["canvas"]["r_quiz_question_id"]
ADV_QUIZ_QUESTION_ID = WEEK["canvas"]["adv_quiz_question_id"]



In [7]:
completion_reports = deduplicate_highest_attempt(create_file_list("submissions", ["LLMCompletionReport.md", f"ass-{ASSIGNMENT_ID}"]))

In [11]:
dat = {
    "student": [],
    "question": [],
    "llm_points": [],
    "llm_explanation": []
}
for completion_report in completion_reports:
    with open(completion_report) as f:
        text = f.read()

    indicators = re.compile(r"Question (#\w+)").findall(text)
    student = [re.compile(r"(stu-\d+)").search(completion_report).group()] * len(indicators)
    
    points = []
    explanations = []
    for indicator in indicators:
        current_text = text[text.find(indicator):len(text)]
        current_text = current_text[current_text.find("grading"):len(current_text)]
        current_text = current_text[current_text.find("Completion Choices"):len(current_text)]
        explanation_text = current_text[current_text.find("<explanation>")+len("<explanation>"):current_text.find("</explanation>")]
        explanations.append(explanation_text)
        current_text = current_text[current_text.find("<points>")+len("<points>"):current_text.find("</points>")]
        points.append(float(current_text))
    
    dat["student"] += student
    dat["question"] += indicators
    dat["llm_points"] += points
    dat["llm_explanation"] += explanations
    

dat = pd.DataFrame(dat)

In [13]:
dat.to_csv(f"grading/week-{WEEK_NUMBER}_llm_grades.csv", index=False)

In [ ]:
dat